In [1]:
import sys,os
import torch
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd()))))))
from models.dVAE.models.model import DiscreteVAE

In [9]:
tensor = torch.zeros(1, 1, 64, 64)

model = DiscreteVAE(
    image_size=64,
    num_tokens=8192,
    codebook_dim=512,
    num_layers=1,
    num_resnet_blocks=2,
    hidden_dim=256,
    channels=1,
)

output = model.get_codebook_indices(tensor)

print(output.shape)

torch.Size([1, 1024])
